In [1]:
import pandas as pd
import os

In [2]:
YEAR = "2020"
VERBOSE = True

In [3]:
COLUMNS_TRANSLATION = {
    "valeur_fonciere": "value_euros",
    "id_parcelle": "id_land",
    "code_postal": "zip_code",
    "nom_commune": "city",
    "type_local": "property_type",
    "adresse_numero": "address_number",
    "nature_mutation": "transaction_type",
    "nature_culture": "land_type", 
    "surface_reelle_bati": "built_surface",
    "surface_terrain": "land_size",
    "nombre_pieces_principales": "rooms_number",  
    "longitude": "longitude",
    "latitude": "latitude"
}

In [4]:
def print_verbose(x):
    if VERBOSE:
        print(x)

# Import dataset and rename columns to keep

In [5]:
df = pd.read_csv(f"full/full_{YEAR}.csv")
df = df[COLUMNS_TRANSLATION.keys()]
df = df.rename(columns=COLUMNS_TRANSLATION)

/home/alexandre/.virtualenvs/opencv_cuda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (10,12,17,18,20,22,35,36) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [6]:
print_verbose(df.dtypes)
print_verbose(df.shape)

value_euros         float64
id_land              object
zip_code            float64
city                 object
property_type        object
address_number      float64
transaction_type     object
land_type            object
built_surface       float64
land_size           float64
rooms_number        float64
longitude           float64
latitude            float64
dtype: object
(827105, 13)


# Drop NA

In [7]:
df = df.dropna(subset=["zip_code", "value_euros", "longitude", "latitude"])

# Deal with locks

## 1. Transaction type

In [8]:
print_verbose(df["transaction_type"].value_counts(dropna=False))

Vente                                 711222
Vente en l'état futur d'achèvement     44198
Echange                                 5235
Vente terrain à bâtir                   1887
Adjudication                             443
Expropriation                              5
Name: transaction_type, dtype: int64


In [9]:
df["transaction_type"] = df["transaction_type"].replace({
    "Vente": "house_or_flat",
    "Vente terrain à bâtir": "other",
})
df = df[df["transaction_type"].isin(["house_or_flat", "other"])]

In [10]:
print_verbose(df["transaction_type"].value_counts(dropna=False))

house_or_flat    711222
other              1887
Name: transaction_type, dtype: int64


## 2. Property type

In [11]:
print_verbose(df["property_type"].value_counts(dropna=False))

NaN                                         264803
Maison                                      170325
Appartement                                 145716
Dépendance                                  104044
Local industriel. commercial ou assimilé     28221
Name: property_type, dtype: int64


In [12]:
df["property_type"] = df["property_type"].replace({
    "Maison": "house",
    "Appartement": "flat"
})
df = df[(df["property_type"].isin(["house", "flat"])) |
        (df["property_type"].isna())]

In [13]:
print_verbose(df["property_type"].value_counts(dropna=False))

NaN      264803
house    170325
flat     145716
Name: property_type, dtype: int64


## 3. Land type

In [14]:
print_verbose(df["land_type"].value_counts(dropna=False))

sols                   224598
NaN                    121917
terres                  65585
prés                    33173
jardins                 28514
terrains a bâtir        20920
taillis simples         18080
terrains d'agrément     17982
landes                  15153
vignes                   7769
futaies résineuses       7244
vergers                  4723
taillis sous futaie      3019
pâtures                  2588
bois                     2330
peupleraies              2195
eaux                     1999
futaies feuillues        1012
prés plantes              608
pacages                   559
herbages                  392
futaies mixtes            202
carrières                 118
chemin de fer              64
prés d'embouche            49
terres plantées            20
landes boisées             20
oseraies                   11
Name: land_type, dtype: int64


In [15]:
df["land_type"] = df["land_type"].replace({
    "sols": "house_or_flat",
    "terrains a bâtir": "land",
})
df = df[(df["land_type"].isin(["house_or_flat", "land"])) |
        (df["land_type"].isna())]

In [16]:
print_verbose(df["land_type"].value_counts(dropna=False))

house_or_flat    224598
NaN              121917
land              20920
Name: land_type, dtype: int64


## 4. Build type

* __Houses__ `'property_type' == 'Maison' && 'transaction_type' == 'Vente' && 'land_type' == 'sols'`
* __Flats__ `'property_type' == 'Appartement' && 'transaction_type' == 'Vente' && 'land_type' == 'sols' or NA`
* __Lands__ `'property_type' is NA && 'land_type' == 'terrains a bâtir'`

In [17]:
df["house_flat_land"] = ""
df.loc[(df["property_type"] == "house") & 
       (df["transaction_type"] == "house_or_flat") &
       (df["land_type"] == "house_or_flat"), "house_flat_land"] = "house"
df.loc[(df["property_type"] == "flat") &
        (df["transaction_type"] == "house_or_flat") &
        ((df["land_type"] == "house_or_flat") | 
         (df["land_type"].isna())), "house_flat_land"] = "flat"
df.loc[(df["property_type"].isna()) &
       (df["land_type"] == "land"), "house_flat_land"] = "land"
df = df[df["house_flat_land"].isin(["house", "flat", "land"])]

In [18]:
print_verbose(df["house_flat_land"].value_counts())

flat     143591
house    135594
land      20131
Name: house_flat_land, dtype: int64


# Format city

In [19]:
df["city"] = df["city"].str.lower()
for old, new in [("-", " "), ("(", ""), (")", ""), ("'", " ")]:
    df["city"] = df["city"].str.replace(old, new, regex=False)
df["city"] = df["city"].map(lambda x: x.split()[0] if "arrondissement" in x else x)

# Add department_code

In [20]:
df["departement_code"] = df["zip_code"] // 1000

# Handle duplicates

In [21]:
df["id_land"].nunique(), df.shape[0]

(232148, 299316)

In [22]:
df.drop_duplicates(subset=["id_land"], keep=False, inplace=True)

# Remove toy columns

In [23]:
COLUMNS_TO_KEEP = [
    "value_euros",
    "zip_code",
    "city",
    "address_number",
    "built_surface",
    "land_size",
    "rooms_number",
    "longitude",
    "latitude",
    "house_flat_land"
]

In [24]:
df = df[COLUMNS_TO_KEEP]

In [25]:
if not os.path.exists("full_clean"):
    os.mkdir("full_clean")

In [26]:
df.to_csv(f"full_clean/full_clean_{YEAR}.csv", index=False)